In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#more imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from pathlib import Path
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix

In [ ]:
path = Path('/kaggle/input/employee-satisfaction-index-dataset/Employee Satisfaction Index.csv')
df = pd.read_csv(path)

emp_id - Unique ID

age - Age

Dept - Department

location - Employee location

education - Employee's education status

recruitment_type - Mode of recruitment

job_level - 1 to 5. The job level of the employee. 1 being the least and 5 being the highest position

rating - 1 to 5. The previous year rating of the employee. 1 being the least and 5 being the highest position

onsite - Has the employee ever went to an onsite location? 0 and 1

awards - No. of awards

certifications - Is the employee certified?

salary - Net Salary

satisfied - Is the employee satisfied with his job?

checking for the head() and null values first

In [ ]:
df.head()

In [ ]:
df.isnull().sum(),df.isna().sum()

dropping unnecessary columns

In [ ]:
df.drop(['Unnamed: 0','emp_id'],axis=1,inplace=True)

In [ ]:
df.Dept.unique(),df.location.unique(),df.recruitment_type.unique()

let's get visual representation of the data

In [ ]:
sns.countplot(x = 'Dept',data = df)

In [ ]:
sns.countplot(x = 'satisfied',data = df,palette='rocket')

In [ ]:
sns.distplot(df['salary'])

In [ ]:
correlation = df.corr()
plt.figure(1,figsize=(10,10))
sns.heatmap(correlation,annot=True)

that's nice destribution right here.

Converting columns to categorical datatype and i won't use one hot encoding for a reason.

In [ ]:
df.Dept.unique(),df.location.unique(),df.recruitment_type.unique(),df.education.unique()

In [ ]:
df.location = df.location.astype('category')
df.Dept = df.Dept.astype('category')
df.recruitment_type =  df.recruitment_type.astype('category')
df.education = df.education.astype('category')

In [ ]:
location_d = dict(enumerate(df.location.cat.categories))
Dept_d = dict(enumerate(df.Dept.cat.categories))
recruitment_type_d = dict(enumerate(df.recruitment_type.cat.categories))
education_d = dict(enumerate(df.education.cat.categories))

In [ ]:
df.location = df.location.cat.codes
df.Dept = df.Dept.cat.codes
df.recruitment_type = df.recruitment_type.cat.codes
df.education = df.education.cat.codes

train\test split

In [ ]:
X = df.drop(['satisfied'],axis=1)
y = df.satisfied

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=0)

**Decision Tree**

In [ ]:
DTmodel = DecisionTreeClassifier().fit(X_train,y_train)

In [ ]:
DTmodel.score(X_test,y_test)

DT gives decent visualization how data splits. 
With random forest we'll get way more variations , but it's usefull to have the initial one.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
tree.plot_tree(DTmodel,fontsize=10,max_depth=3)

In [ ]:
treedata = tree.export_graphviz(DTmodel,out_file=None,feature_names=X_train.columns,filled=True,rounded=True,special_characters=True,max_depth=4)
graph = graphviz.Source(treedata)
graph

**Random Forest Classifier**

In [ ]:
RFmodel = RandomForestClassifier().fit(X_train,y_train)

In [ ]:
RFmodel.score(X_test,y_test)

Slightly better improvement. Score is low for basic binary classification, probably due to even distribution of generated satisfaction points. It would be different for sure with real-world data.

Anyway, let's finish notebook with confusion matrix

In [ ]:
test_pred = RFmodel.predict(X_test)
cm = confusion_matrix(y_test,test_pred)
cm

and feature importance

In [ ]:
n_features = X.shape[1]
plt.barh(range(n_features),RFmodel.feature_importances_)
plt.yticks(np.arange(n_features),df.columns[1:])